# WI-Projekt
Nr. 18

... <br>
... <br>
BA.WIN.22HS.VZa

... <br>
... <br>
BA.WIN.22HS.VZb

Pascal Kern <br>
kernpas1@students.zhaw.ch <br>
BA.WIN.22HS.VZa

## Setup

#### Get required Packages

In [5]:
import json
import os

from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

credentials = "credentials.json"

with open(credentials, 'r') as f:
    credentials = json.load(f)

# API-Schlüssel setzen
os.environ["OPENAI_API_KEY"] = credentials['openai']['api_key'] # OpenAI

#### Connect to DB

In [6]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'AntÃ´nio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

### Agent OpenAI

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0) ## Which Large Language model are we using?

OpenAI_agent = create_sql_agent(
    llm, 
    toolkit= SQLDatabaseToolkit(llm=llm, db=db), 
    agent_type='tool-calling', 
    verbose=True) 


### Execute

In [8]:
OpenAI_agent.invoke(
    "Welcher Artist hat am meisten Albums?"
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Album, Artist'}`



CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/
Invoking: `sql_db_query_checker` with `{'query': 'SELECT Artist.Name, COUNT(Album.AlbumId) AS AlbumCount FROM Artist JOIN Album ON Artist.ArtistId = Album.ArtistId GROUP BY Artist.ArtistId ORDER BY AlbumCo

{'input': 'Welcher Artist hat am meisten Albums?',
 'output': 'Der Artist mit den meisten Alben ist "Iron Maiden" mit 21 Alben.'}